In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import math

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna 

# Feel free to add all the libraries you need
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import optuna.visualization as vis

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from geopy.distance import geodesic

In [20]:
df1 = pd.read_csv('data/labelencoded.csv', index_col=0)

In [31]:
# Create the seasons column
df1.loc[(df1['target'] ==0 ), 'target_bin'] = 0
df1.loc[(df1['target'] > 0) & (df1['target'] < 10), 'target_bin'] = 1 
df1.loc[(df1['target'] >= 10) & (df1['target'] < 100), 'target_bin'] = 2
df1.loc[(df1['target'] >= 100) & (df1['target'] < 1000), 'target_bin'] = 3
df1.loc[(df1['target'] >= 1000), 'target_bin'] = 4

In [32]:
df = df1.drop(['target'], axis=1)

In [34]:
# define features and target
features = df.columns.tolist()
features.remove('target_bin')
X = df[features]
y = df['target_bin']

# LETS DO FEATURE ENGINEERING.....gonna square some shit 
# generate interaction features to the 3rd degree
p = PolynomialFeatures(interaction_only=False, include_bias=False, degree=1)
X_poly = p.fit_transform(X)
p.n_output_features_

27

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state=12, test_size=0.5)

In [38]:
# Creating an Objective Function
def KNN_optimzization():
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[]}
    ridge_opt_dictionary = { 'alpha': [], 'intercept' : [], 'tol': [], 'solver':[], 'no_of_coefficients':[]}

    def objective_knn(trial):
        
        # Define hyperparameters
        param_knn = {
            
            'n_neighbors' : trial.suggest_int('n_neighbors', 1, 100),
            'weights' : trial.suggest_categorical('weights', ['uniform', 'distance']),
            'metric' : trial.suggest_categorical('metric', ['euclidean', 'manhattan','minkowski']),
            'p' : trial.suggest_int('p', 1, 50),
            'algorithm' : trial.suggest_categorical('algorithm',['auto','ball_tree', 'kd_tree', 'brute']),
            'leaf_size' : trial.suggest_int('leaf_size', 1, 100)
            
            }

        print(param_knn)
        
        # Create and train the model
        knn = KNeighborsClassifier(**param_knn)
        knn.fit(X_train, y_train)

        # Evaluate the model
        score = cross_val_score(knn, X_train, y_train, cv=3, scoring="accuracy").mean()
        

        return score
    study_knn = optuna.create_study(direction='maximize',study_name = "KNN")
    study_knn.optimize(objective_knn, n_trials=50)

In [39]:
KNN_optimzization()

[I 2023-09-01 08:56:21,095] A new study created in memory with name: KNN


{'n_neighbors': 54, 'weights': 'distance', 'metric': 'manhattan', 'p': 5, 'algorithm': 'kd_tree', 'leaf_size': 50}


[I 2023-09-01 08:56:26,825] Trial 0 finished with value: 0.5602233103345946 and parameters: {'n_neighbors': 54, 'weights': 'distance', 'metric': 'manhattan', 'p': 5, 'algorithm': 'kd_tree', 'leaf_size': 50}. Best is trial 0 with value: 0.5602233103345946.


{'n_neighbors': 38, 'weights': 'distance', 'metric': 'manhattan', 'p': 33, 'algorithm': 'kd_tree', 'leaf_size': 49}


[I 2023-09-01 08:56:30,731] Trial 1 finished with value: 0.5598709103049188 and parameters: {'n_neighbors': 38, 'weights': 'distance', 'metric': 'manhattan', 'p': 33, 'algorithm': 'kd_tree', 'leaf_size': 49}. Best is trial 0 with value: 0.5602233103345946.


...

{'n_neighbors': 11, 'weights': 'uniform', 'metric': 'manhattan', 'p': 38, 'algorithm': 'auto', 'leaf_size': 92}


[I 2023-09-01 09:02:34,664] Trial 17 finished with value: 0.5693671637361821 and parameters: {'n_neighbors': 11, 'weights': 'uniform', 'metric': 'manhattan', 'p': 38, 'algorithm': 'auto', 'leaf_size': 92}. Best is trial 10 with value: 0.5701461532754655.


{'n_neighbors': 29, 'weights': 'uniform', 'metric': 'minkowski', 'p': 45, 'algorithm': 'auto', 'leaf_size': 66}


[W 2023-09-01 09:04:33,052] Trial 18 failed with parameters: {'n_neighbors': 29, 'weights': 'uniform', 'metric': 'minkowski', 'p': 45, 'algorithm': 'auto', 'leaf_size': 66} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\erick\gitrepositories\flight_delays_ML\.venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\erick\AppData\Local\Temp\ipykernel_15880\2551390458.py", line 28, in objective_knn
    score = cross_val_score(knn, X_train, y_train, cv=3, scoring="accuracy").mean()
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\erick\gitrepositories\flight_delays_ML\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 515, in cross_val_score
    cv_results = cross_validate(
                 ^^^^^^^^^^^^^^^
  File "c:\Users\erick\gitrepositories\flight_delays_ML\.venv\

KeyboardInterrupt: 